<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Simple Decision Prompts</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundamental building blocks for eliciting effective responses from AI models. This module demonstrates how to use common prompt engineering techniques within Semantic Kernel. If you’ve used ChatGPT or Microsoft Copilot, you’re already familiar with prompting: given an instruction, a language model predicts the most likely and relevant response. With Semantic Kernel, you can build applications, services, and APIs that execute these prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal is to maximize the quality and clarity of the AI’s output, often by using specific wording, added context, or concrete examples within the prompt.

By combining Decision Intelligence with Prompt Engineering, you can create “Generative AI Decision Intelligence.” This approach leverages GenAI models to reason through complex tasks, gather intelligence, recommend decisions, and communicate results more effectively.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connection info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.55"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.55.0

Using Azure OpenAI Service


### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [3]:
// A Decision Intelligence prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

General Formatting Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
var response = simplePromptResponse.GetValue<string>(); 

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

### 1. SWOT Analysis  
**Description:**  
SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This framework encourages individuals or organizations to systematically evaluate internal and external factors that can influence the outcome of a decision.  
**How it Enhances Decisions:**  
By clearly mapping out positives and negatives (both internal and external), users can better understand their position, anticipate challenges, and leverage strengths to capitalize on opportunities.

### 2. Cost-Benefit Analysis (CBA)  
**Description:**  
CBA involves identifying and quantifying the costs and expected benefits of various options to determine which choice offers the greatest net benefit.  
**How it Enhances Decisions:**  
This framework supports logical, objective comparison by translating consequences into concrete values, making it easier to justify and communicate decisions.

### 3. The Eisenhower Matrix  
**Description:**  
This model helps prioritize tasks or choices by categorizing them based on urgency and importance, creating four quadrants: urgent/important, not urgent/important, urgent/not important, and not urgent/not important.  
**How it Enhances Decisions:**  
By distinguishing between what’s urgent and what’s important, users can focus on high-impact activities and avoid being sidetracked by less significant tasks.

### 4. Decision Tree Analysis  
**Description:**  
A decision tree is a visual map of possible decisions and their potential outcomes, including risks, rewards, and uncertainties.  
**How it Enhances Decisions:**  
It clarifies complex choices by explicitly laying out possible paths, supporting probabilistic reasoning and identifying optimal courses of action in scenarios with multiple variables.

### 5. Six Thinking Hats  
**Description:**  
This creative framework by Edward de Bono encourages examining a problem from six distinct perspectives (facts, emotions, negatives, positives, creativity, and control) by metaphorically "wearing" different colored hats.  
**How it Enhances Decisions:**  
By deliberately exploring a variety of viewpoints, this method fosters balanced thinking, prevents groupthink, and leads to more comprehensive analyses and innovative solutions.

### Step 3 - Execute a Decision Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> 📝 **Note:** An average human can read between 25-40 Tokens / second. Therefore, while streaming certainly helps with providing AI output to the user, it begins to lose its effectiveness at large token velocity.  

In [4]:
// Decision Prompt using Streaming output chunks 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

### 1. SWOT Analysis

**Description:**  
SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This framework encourages users to systematically evaluate internal and external factors affecting a decision or project.  
**How it supports better analysis and reasoning:**  
By categorizing influences into these four areas, SWOT helps ensure that decisions consider both advantages and risks, from within and outside the organization. It fosters a balanced view and helps to identify strategic actions.

---

### 2. Decision Matrix (Weighted Scoring Model)

**Description:**  
A decision matrix helps compare multiple options by listing criteria, assigning weights to each criterion, and scoring each option accordingly.  
**How it supports better analysis and reasoning:**  
It introduces objectivity and quantifies preferences, making it easier to compare complex options. This method is particularly useful where decisions involve many competing factors.

---

### 3. Cost-Benefit Analy

### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can output Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting output. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [5]:
// A new decision prompt to help with describing decision-making frameworks using Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat);
var response = simplePromptResponse.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Decision-Making Framework      | Description                                                                                      | How It Enhances Analysis & Reasoning                                             |
|-------------------------------|--------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------|
| SWOT Analysis                 | Assesses Strengths, Weaknesses, Opportunities, and Threats.                                      | Encourages balanced evaluation of internal and external factors affecting a decision.           |
| Cost-Benefit Analysis         | Compares the costs and anticipated benefits of different options.                                | Supports a quantitative approach to weighing alternatives for optimal choices.    |
| Decision Matrix (Prioritization Grid) | Rates and compares multiple options using weighted criteria.                                        | Facilitates objective, systematic evaluation when many variables are involved.    |
| Six Thinking Hats             | Explores problems from six distinct perspectives (logical, emotional, creative, etc.).           | Improves comprehensive thinking and minimizes cognitive biases.                   |
| OODA Loop                     | Stands for Observe, Orient, Decide, Act; a cyclical process for rapid decision-making.           | Promotes continual situational assessment and agile responses in dynamic settings.|

### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Temperature or other settings.

> **📝 Note:** The supported prompt settings are dependent on the API plus the specific model version. For example, an AI model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available. Types of model execution (general versus reasoning) will also have different execution setting parameters. 

In [7]:
// Create a new OpenAI Prompt Execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Create a new KernelArguments object with the OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseWithOpenAI = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat, kernelArguments);
var response = promptResponseWithOpenAI.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework                | Description                                                                                              | How It Enhances Analysis & Reasoning                                                                                 |
|--------------------------|----------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis            | Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.            | Encourages comprehensive internal and external assessment, ensuring balanced consideration of all relevant factors.   |
| Cost-Benefit Analysis    | Compares the costs and benefits of different options quantitatively or qualitatively.                     | Supports objective evaluation by quantifying trade-offs, aiding in rational and economically sound decisions.         |
| Decision Matrix          | Ranks options against defined criteria using a scoring system.                                            | Structures complex choices, reduces bias, and clarifies priorities by making criteria and scoring explicit.           |
| Six Thinking Hats        | Involves looking at a problem from six distinct perspectives (logic, emotion, caution, optimism, etc.).   | Promotes holistic thinking, creativity, and critical analysis by systematically exploring all angles of a decision.   |
| OODA Loop                | Stands for Observe, Orient, Decide, Act; a cyclical process for rapid and adaptive decision-making.       | Enhances situational awareness and adaptability, allowing for iterative improvement and timely responses.             |

### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decision Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific (i.e. decision intelligence)
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [ ]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);


var promptResponseWithTemplate = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt, kernelArguments);
var response = promptResponseWithTemplate.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

### 1. Pros and Cons List

**Description:**  
List the advantages (pros) and disadvantages (cons) of each option. Compare the lists to see which option has more or stronger positives.

**How to Use:**  
- Write down your options.
- For each option, list the pros and cons.
- Weigh the importance of each point if needed.
- Choose the option with the most compelling pros and least impactful cons.

**Best For:**  
Simple choices (e.g., what to eat, which route to take, whether to accept an invitation).

---

### 2. Eisenhower Matrix (Urgent-Important Matrix)

**Description:**  
Sort tasks or decisions based on their urgency and importance to prioritize what to do next.

**How to Use:**  
- Draw a 2x2 grid: Urgent/Not Urgent vs. Important/Not Important.
- Place each task or decision in the appropriate quadrant.
- Focus on Important & Urgent first, then Important & Not Urgent.

**Best For:**  
Time management, prioritizing daily tasks, handling multiple responsibilities.

---

### 3. 2-Minute Rule

**Description:**  
If a decision or task can be completed in two minutes or less, do it immediately. Otherwise, decide whether to schedule, delegate, or drop it.

**How to Use:**  
- Ask: Can I resolve this in 2 minutes?
- If yes, do it now.
- If no, decide when/how to handle it.

**Best For:**  
Quick decisions, handling small tasks, reducing procrastination.

---

### 4. OODA Loop (Observe, Orient, Decide, Act)

**Description:**  
A cyclical process for making decisions quickly and effectively, especially in dynamic situations.

**How to Use:**  
- **Observe:** Gather information about the situation.
- **Orient:** Interpret what you’ve observed.
- **Decide:** Choose a course of action.
- **Act:** Implement your decision.
- Repeat as needed.

**Best For:**  
Situations requiring quick thinking, adapting to new information, everyday problem-solving.

---

### 5. 10-10-10 Rule

**Description:**  
Consider the consequences of your decision in three time frames: 10 minutes, 10 months, and 10 years.

**How to Use:**  
- Ask: How will I feel about this decision in 10 minutes? 10 months? 10 years?
- Reflect on the short-, medium-, and long-term impacts.
- Choose the option that aligns best with your values and goals over time.

**Best For:**  
Decisions with emotional weight, long-term consequences, or when you’re tempted by short-term gains.

---

### Summary Table

| Framework           | Best For                                 | Key Steps                                    |
|---------------------|------------------------------------------|----------------------------------------------|
| Pros and Cons List  | Everyday choices                         | List, weigh, compare                         |
| Eisenhower Matrix   | Prioritizing tasks                       | Sort by urgency/importance                   |
| 2-Minute Rule       | Quick decisions, small tasks             | Do now if ≤2 min, else plan                  |
| OODA Loop           | Dynamic, evolving situations             | Observe, Orient, Decide, Act, repeat         |
| 10-10-10 Rule       | Emotional/long-term impact decisions     | Reflect on 10 min, 10 months, 10 years ahead |

These frameworks are simple, flexible, and can be applied to a wide range of daily decisions.

### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full **Decision Intelligence** framework will not be used, rather a simple request for Artificial Intelligence to recommend a path forward (recommendation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [ ]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed Decision Scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommendation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, // Increase the max tokens to allow for a more detailed response
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseScenario = await semanticKernel.InvokePromptAsync(scenarioDecisionPromptTemplate, kernelArguments);
var response = promptResponseScenario.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

### Recommendation Table: Direct Enrollment in Four-Year University vs. Starting at Community College

| Decision Factor                  | Four-Year University (Direct Enrollment)           | Community College (Associate First)               | Analysis & Recommendation                                                                                 |
|----------------------------------|---------------------------------------------------|---------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| **Financial Considerations**     | High cost ($50k-$60k/year); some loans required   | Low cost ($3k-$5k/year); significant savings      | Community college offers substantial savings, but family can afford university with manageable loans.     |
| **Career and Major Uncertainty** | Risk of extra time/cost if switching majors; less flexibility to explore | Greater flexibility to explore majors at lower cost; possible transfer challenges | Community college allows low-cost exploration, reducing risk if Alex changes her mind about her major.    |
| **Academic Consistency & Networking** | Early access to campus resources, professors, internships, and networking | Delayed access to four-year opportunities; may need to adjust after transfer | Four-year university provides earlier, deeper networking and professional opportunities.                  |
| **Future Impact**                | Builds network and experience sooner; higher debt risk | Lower debt; may delay network-building and experience | Four-year university supports Alex’s long-term career/network goals, but increases financial risk.        |
| **Personal Preferences**         | Prefers early professional networking; not focused on social life | Comfortable with delayed campus experience; values flexibility | Alex values professional networking and experience more than immediate cost savings.                      |

### Final Recommendation

| **Recommended Path**              | **Rationale**                                                                                                      |
|-----------------------------------|--------------------------------------------------------------------------------------------------------------------|
| **Direct Enrollment in Four-Year University** | While community college offers significant cost savings and flexibility for exploring majors, Alex’s priorities—early professional networking, access to campus resources, and building a strong foundation for her career—are best served by starting at the four-year university. The family can manage the cost with some loans, and the long-term benefits of early integration into her chosen field and network outweigh the medium-level financial concerns. Alex should proactively use campus resources to explore majors in her first two years, minimizing the risk of extended time or extra cost if she changes direction. |